In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [2]:
from django.db import connection
connection.connect()

In [3]:
from fotoweb.models import *

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

class SelDriver():
  def __init__(self,dbg=False):
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    if dbg:
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    #Change chrome driver path accordingly
    chrome_driver = "d:\chromedriver-win64\chromedriver.exe"
    service = Service(chrome_driver)
    self.driver = webdriver.Chrome(service=service, options=chrome_options)
    print (self.driver)
    self.driver.implicitly_wait(5)
    self.vars = {}

In [6]:
driver=SelDriver(True)

<selenium.webdriver.chrome.webdriver.WebDriver (session="a34cac6a62ca965810af88639227baea")>


In [7]:

#ee=driver.driver.find_elements(By.CSS_SELECTOR, '.cm-media-thumb img')
ee=driver.driver.find_elements(By.CSS_SELECTOR, '.cm-media-thumb img')
ee

[<selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="9399DE47AB1DF1FD9684D0D7B4536741_element_390")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="9399DE47AB1DF1FD9684D0D7B4536741_element_391")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="9399DE47AB1DF1FD9684D0D7B4536741_element_392")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="9399DE47AB1DF1FD9684D0D7B4536741_element_393")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="9399DE47AB1DF1FD9684D0D7B4536741_element_394")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="9399DE47AB1DF1FD9684D0D7B4536741_element_395")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88

In [8]:
srcs=[]
for e in ee:
    src=e.get_attribute('src')
    r=re.search(r'(\d+)\.jpg',src)
    print(src,r.group(1))
    srcs.append(r.group(1))



https://image.shutterstock.com/image-photo/vibrant-autumn-foliage-east-lothian-250nw-2392439471.jpg 2392439471
https://image.shutterstock.com/image-photo/vibrant-autumn-landscape-east-lothian-250nw-2392439483.jpg 2392439483
https://image.shutterstock.com/image-photo/capturing-tranquility-autumn-vibrant-yellow-250nw-2392439551.jpg 2392439551
https://image.shutterstock.com/image-photo/golden-sunlight-filtering-through-colorful-250nw-2392439637.jpg 2392439637
https://image.shutterstock.com/image-photo/serene-autumn-landscape-east-lothian-250nw-2392439231.jpg 2392439231
https://image.shutterstock.com/image-photo/vibrant-autumn-leaves-east-lothian-250nw-2392439317.jpg 2392439317
https://image.shutterstock.com/image-photo/autumn-masterpiece-vibrant-leaves-contrasting-250nw-2392439383.jpg 2392439383
https://image.shutterstock.com/image-photo/vibrant-autumn-colors-scenic-landscape-250nw-2392438931.jpg 2392438931
https://image.shutterstock.com/image-photo/tranquil-countryside-setting-diverse-ac

In [9]:
images=[]
for src in srcs:
    driver.driver.get('https://submit.shutterstock.com/catalog_manager/images/'+src)
    name = driver.driver.find_element(By.CSS_SELECTOR,'.o_List_List_itemText div').text
    description = driver.driver.find_element(By.CSS_SELECTOR,'.o_EditorDescription_EditorDescription_descriptionInput').text
    images.append({
        'name':name,
        'description':description,
        'shutter-id':src
    })
    i=None
    try:
        i=Image.objects.get(name=name)
        print(name,'found exact match')
    except:
        name = driver.driver.find_element(By.CSS_SELECTOR,'.o_List_List_itemText div').text.replace('.jpg','')
        try:
            i=Image.objects.filter(name__contains=name).first()
            print(name,'found by contains')
        except:
            print(name,'NOT found')

    if i and i.shutter:
        print('already shutter')
    elif i:
        i.shutter_dt=i.updated_dt
        if not i.title:
            i.title=driver.driver.find_element(By.CSS_SELECTOR,'.o_EditorDescription_EditorDescription_descriptionInput').text
        i.shutter=True
        i.shutter_url='https://submit.shutterstock.com/catalog_manager/images/'+src
        i.save()
        print('shutter updated')


23-11-03-DSC04573-1.jpg found exact match
shutter updated
23-11-03-DSC04587-1.jpg found exact match
shutter updated
23-11-09-DSC04631-1.jpg found exact match
shutter updated
23-11-09-DSC04638-1.jpg found exact match
shutter updated
23-11-09-DSC04636-1.jpg found exact match
shutter updated
23-10-12-DSC04474-1.jpg found exact match
shutter updated
23-10-22-DSC04493-1.jpg found exact match
shutter updated
23-11-03-DSC04591-1.jpg found exact match
shutter updated
23-11-03-DSC04596-1.jpg found exact match
shutter updated
23-11-09-DSC04618-1.jpg found exact match
shutter updated
23-11-09-DSC04639-1.jpg found exact match
shutter updated
23-11-03-DSC04589-1.jpg found exact match
shutter updated
23-10-31-DSC04551-1.jpg found exact match
shutter updated
23-11-09-DSC04632-1.jpg found exact match
shutter updated
23-10-22-DSC04500-1.jpg found exact match
shutter updated
23-10-31-DSC04549-1.jpg found exact match
shutter updated
23-11-09-DSC04634-1.jpg found exact match
shutter updated
23-10-22-DSC04

In [ ]:
images

In [10]:
driver.driver.get("https://submit.shutterstock.com/en/data_catalog")

In [11]:
cards = driver.driver.find_elements(By.CSS_SELECTOR, ".MuiGrid-root > .MuiBox-root > .MuiPaper-root")
cards

[<selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="7B22C0CD513E0E22251AC315DE05608A_element_74")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="7B22C0CD513E0E22251AC315DE05608A_element_75")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="7B22C0CD513E0E22251AC315DE05608A_element_76")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="7B22C0CD513E0E22251AC315DE05608A_element_77")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="7B22C0CD513E0E22251AC315DE05608A_element_78")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227baea", element="7B22C0CD513E0E22251AC315DE05608A_element_79")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a34cac6a62ca965810af88639227

In [12]:
for c in cards:
    #print(c.text)
    idiv = c.find_element(By.CSS_SELECTOR,".MuiBox-root:nth-child(1) > .MuiTypography-root")
    name = idiv.text
    print(name)

    i=None
    try:
        i=Image.objects.get(name=name)
        print(name,'found exact match')
    except:
        print(name,'NOT found')

    if i and i.shutter:
        print('already shutter')
    elif i:
        i.shutter_dt=i.updated_dt
        i.shutter=True
        i.shutter_url='Data licensing'
        i.save()
        print('shutter updated')

                       

23-11-12-DSC04702-1.jpg
23-11-12-DSC04702-1.jpg found exact match
shutter updated
23-08-31-DSC03822-1.jpg
23-08-31-DSC03822-1.jpg found exact match
shutter updated
23-08-31-DSC03818-1.jpg
23-08-31-DSC03818-1.jpg found exact match
already shutter
23-08-31-DSC03855-1.jpg
23-08-31-DSC03855-1.jpg found exact match
already shutter
23-09-22-DSC04051-1.jpg
23-09-22-DSC04051-1.jpg found exact match
already shutter
23-08-31-DSC03807-1.jpg
23-08-31-DSC03807-1.jpg found exact match
already shutter
23-08-31-DSC03814-1.jpg
23-08-31-DSC03814-1.jpg found exact match
already shutter
23-08-31-DSC03778-1.jpg
23-08-31-DSC03778-1.jpg found exact match
already shutter
23-08-31-DSC03788-1.jpg
23-08-31-DSC03788-1.jpg found exact match
already shutter
23-08-31-DSC03797-1.jpg
23-08-31-DSC03797-1.jpg found exact match
already shutter
23-08-31-DSC03779-1.jpg
23-08-31-DSC03779-1.jpg found exact match
already shutter
23-08-31-DSC03773-1.jpg
23-08-31-DSC03773-1.jpg found exact match
already shutter
23-03-03-DJI_003